In [22]:
import csv

def filter_churn_retention(input_file_path, output_file_path, word_to_check):
    """
    Funktsioon klientide jaotamiseks kahte faili: "churn.csv" kliendid, kes on lõpetanud koostöö ettevõttega, ja "retention.csv" kliendid, kes jätkuvalt kasutavad ettevõtte teenuseid.
    """
    with open(input_file_path, mode='r', encoding='utf-8') as infile, open(output_file_path, mode='w', encoding='utf-8', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        for client in reader:
            if client[-1] == word_to_check:
                writer.writerow(client)

input_file = 'initial.csv'
output_file_churn = 'churn.csv'
output_file_retention = 'retention.csv'
churn = 'Yes'
retention = 'No'

filter_churn_retention(input_file, output_file_churn, churn)
filter_churn_retention(input_file, output_file_retention, retention)

In [23]:
def count_services_of_one_client(client):
    """
    Arvutame iga kliendi kasutatavate teenuste hulka.
    """
    cells = client.split(',')
    services = cells[6:15]
    number_of_services = sum(1 for service in services if "No" not in service)
    return number_of_services

def count_partner_dependents_of_one_client(client):
    """
    Arvutame iga kliendi partnerite/ülalpeetavate hulka.
    """
    cells = client.split(',')
    partners_or_dependents = cells[3:5]
    number_of_partners_or_dependents = sum(1 for partner_dependent in partners_or_dependents if "No" not in partner_dependent)
    return number_of_partners_or_dependents

def count_services_and_partner_dependents_total(file):
    """
    Loeme kokku failis olevate klientide kasutatavate teenuste ja partnerite/ülalpeetavate arvu, et leida iga teguri kohta keskmine väärtus. Tagastame mõlema keskmise - teenuste arvu ja partnerite/ülalpeetavate arvu - ühe stringina. Pärast salvestame mõlema faili kohta  andmed lõppfaili 'file_to_analyze.txt'.
    """
    total_clients = 0
    total_services = 0
    total_partners_or_dependents = 0
    with open(file, 'r') as file:
        for client in file:
            total_services += count_services_of_one_client(client)
            total_partners_or_dependents += count_partner_dependents_of_one_client(client)
            total_clients += 1
            
    average_number_of_services = round(total_services / total_clients, 1)
    average_number_of_partners_or_dependents = round(total_partners_or_dependents / total_clients, 1)
    
    result = 'Keskmine teenuste arv inimese kohta: ' + str(average_number_of_services) + ' ja keskmine partnerite/ülalpeetavate arv: ' + str(average_number_of_partners_or_dependents)
    
    return result

churn = 'churn.csv'
retention = 'retention.csv'

with open('file_to_analyze.txt', mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Säilitatud kliendid: ' + count_services_and_partner_dependents_total(retention)])
    writer.writerow(['Lahkunud kliendid: ' + count_services_and_partner_dependents_total(churn)])

In [24]:
def count_contract_types(csv_file_path):
    """
    Iga lepingutüübi jaoks arvutame failis olevate klientide protsendi, kes kasutavad antud lepingut. Väljastame informatsiooni stringi kujul. Pärast salvestame mõlema faili kohta andmed lõppfaili 'file_to_analyze.txt'.
    """
    contract_counts = {"One year": 0, "Month-to-month": 0, "Two year": 0}
    contract_percents = {}
    total_rows = 0

    with open(csv_file_path, 'r') as file:
        for line in file:
            total_rows += 1
            cells = line.split(',')
            contract_type = cells[15]
            if contract_type in contract_counts:
                contract_counts[contract_type] += 1
    result = ''
    for key in contract_counts:
        contract_percents[key] = (contract_counts[key] / total_rows) * 100
        result = result + key + ': ' + str(round((contract_counts[key] / total_rows) * 100, 1)) + '% '
    return result

churn = 'churn.csv'
retention = 'retention.csv'

with open('file_to_analyze.txt', mode='a', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Säilitatud klientide lepingutüüpide protsendid: " + str(count_contract_types(retention))])
    writer.writerow(["Lahkunud  klientide lepingutüüpide protsendid: " + str(count_contract_types(churn))])

In [25]:
def calculate_average_monthly_charge(file_path):
    """
    Funktsioon arvutab antud failis oleva kliendi keskmise kuumakse. Pärast salvestame mõlema faili kohta  andmed lõppfaili 'file_to_analyze.txt'.
    """
    total_sum = 0
    line_count = 0
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for client in reader:
            number = float(client[-3])
            total_sum += number
            line_count += 1
    
    return round(total_sum / line_count, 1)

churn = 'churn.csv'
retention = 'retention.csv'

average_churn = calculate_average_monthly_charge(churn)
average_retention = calculate_average_monthly_charge(retention)

with open('file_to_analyze.txt', mode='a', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Säilitatud klientide keskmine kuumakse: " + str(average_retention) + '€'])
    writer.writerow(["Lahkunud klientide keskmine kuumakse:: " + str(average_churn) + '€'])

In [21]:
def find_potential_churn(input_file_path, output_file_path,):
    """
    Funktsioon analüüsib, kas klient kuulub potentsiaalselt lahkuvate klientide hulka, lähtudes varasemast analüüsist. Kui klient vastab potentsiaalse lahkumise kriteeriumitele, salvestatakse tema andmed faili 'potential.csv'. Input fail on 'retention.csv', kus on kõik säilitatud kliendid.
    """
    with open(input_file_path, mode='r', encoding='utf-8') as infile, open(output_file_path, mode='w', encoding='utf-8', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        for client in reader:
            services = client[6:15]
            services_num = sum(1 for part in services if "No" not in part)
            partners_or_dependents = client[3:5]
            partner_dependents_num = sum(1 for part in partners_or_dependents if "No" not in part)
            contract_type = client[15]
            monthly_charge = float(client[-3])
            if services_num <= 4.1 and partner_dependents_num <= 1 and contract_type == 'Month-to-month' and monthly_charge > 74.4:
                writer.writerow(client)


retention = 'retention.csv'
potential = 'potential.csv'
find_potential_churn(retention, potential)